# Data preparation

In this notebook we prepare coefficients for the Portfolio optimization problem in mean-variance statement. The risk is calculated as a covariation matrix of the chosen assets, and the return is calculated as a mean value of day returns of an asset. The data is collected over the trading year (251 day). 

In [47]:
from datetime import datetime
from concurrent import futures
import pandas as pd
from pandas import DataFrame
from pathlib import Path
import os
from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()

In [48]:
""" list of s_anp_p companies """
s_and_p = [
    'MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 
    'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT',
    'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AAPL', 'AMAT', 'APTV', 'ADM',
    'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BBWI', 'BAX', 'BDX', 'BRK.B',
    'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'CHRW',
    'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY',
    'CERN', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX',
    'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CTRA',
    'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DISCA',
    'DISCK', 'DISH', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE', 'DUK', 'DRE', 'DD', 'DXC', 'EMN', 'ETN', 'EBAY', 'ECL',
    'EIX', 'EW', 'EA', 'EMR', 'ENPH', 'ETR', 'EOG', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY', 'EVRG', 'ES', 'RE', 'EXC',
    'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FB', 'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FE', 'FRC', 'FISV', 'FLT', 'FMC', 'F',
    'FTNT', 'FTV', 'FBHS', 'FOXA', 'FOX', 'BEN', 'FCX', 'GPS', 'GRMN', 'IT', 'GNRC', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD',
    'GL', 'GPN', 'GS', 'GWW', 'HAL', 'HBI', 'HIG', 'HAS', 'HCA', 'PEAK', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HOLX', 'HD',
    'HON', 'HRL', 'HST', 'HWM', 'HPQ', 'HUM', 'HBAN', 'HII', 'IEX', 'IDXX', 'INFO', 'ITW', 'ILMN', 'INCY', 'IR', 'INTC', 'ICE',
    'IBM', 'IP', 'IPG', 'IFF', 'INTU', 'ISRG', 'IVZ', 'IPGP', 'IQV', 'IRM', 'JKHY', 'J', 'JBHT', 'SJM', 'JNJ', 'JCI', 'JPM',
    'JNPR', 'KSU', 'K', 'KEY', 'KEYS', 'KMB', 'KIM', 'KMI', 'KLAC', 'KHC', 'KR', 'LHX', 'LH', 'LRCX', 'LW', 'LVS', 'LEG','LDOS',
    'LEN', 'LLY', 'LNC', 'LIN', 'LYV', 'LKQ', 'LMT', 'L', 'LOW', 'LUMN', 'LYB', 'MTB', 'MRO', 'MPC', 'MKTX', 'MAR', 'MMC',
    'MLM', 'MAS', 'MA', 'MTCH', 'MKC', 'MCD', 'MCK', 'MDT', 'MRK', 'MET', 'MTD', 'MGM', 'MCHP', 'MU', 'MSFT', 'MAA','MRNA',
    'MHK', 'TAP', 'MDLZ', 'MPWR', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'NDAQ', 'NTAP', 'NFLX', 'NWL', 'NEM', 'NWSA',
    'NWS', 'NEE', 'NLSN', 'NKE', 'NI', 'NSC', 'NTRS', 'NOC', 'NLOK', 'NCLH', 'NRG', 'NUE', 'NVDA', 'NVR', 'NXPI','ORLY', 'OXY',
    'ODFL', 'OMC', 'OKE', 'ORCL', 'OGN', 'OTIS', 'PCAR', 'PKG', 'PH', 'PAYX', 'PAYC', 'PYPL', 'PENN', 'PNR', 'PBCT', 'PEP',
    'PKI', 'PFE', 'PM', 'PSX', 'PNW', 'PXD', 'PNC', 'POOL', 'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PTC', 'PEG', 'PSA',
    'PHM', 'PVH', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTX', 'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RHI', 'ROK', 'ROL',
    'ROP', 'ROST', 'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SEE', 'SRE', 'NOW', 'SHW', 'SPG', 'SWKS', 'SNA', 'SO', 'LUV',
    'SWK', 'SBUX', 'STT', 'STE', 'SYK', 'SIVB', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW', 'TTWO', 'TPR', 'TGT', 'TEL', 'TDY', 'TFX',
    'TER', 'TSLA', 'TXN', 'TXT', 'TMO', 'TJX', 'TSCO', 'TT', 'TDG', 'TRV', 'TRMB', 'TFC', 'TWTR', 'TYL', 'TSN', 'UDR', 'ULTA',
    'USB', 'UAA', 'UA', 'UNP', 'UAL', 'UNH', 'UPS', 'URI', 'UHS', 'VLO', 'VTR', 'VRSN',     'VRSK', 'VZ', 'VRTX', 'VFC', 'VIAC',
    'VTRS', 'V', 'VNO', 'VMC', 'WRB', 'WAB', 'WMT', 'WBA', 'DIS', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WST', 'WDC', 'WU', 'WRK',
    'WY', 'WHR', 'WMB', 'WLTW', 'WYNN', 'XEL', 'XLNX', 'XYL', 'YUM', 'ZBRA', 'ZBH','ZION', 'ZTS'
]

bad_names = []

In [49]:
dirpath = os.path.join(Path().resolve(), 'S&P_500')
Path(dirpath).mkdir(parents=True, exist_ok=True)

The time, over which a trading data is collected. We are restricted by one trading year. Additionally, the example of how the data looks like in csv file, is shown. 

In [51]:
now_time = datetime(now_time.year, now_time.month , now_time.day-1)
start_time = datetime(now_time.year - 1, now_time.month , now_time.day-1)
print(start_time, now_time)

data = pdr.get_data_yahoo('AAPL', start_time, now_time)
data

2022-02-25 00:00:00 2023-02-26 00:00:00
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-25,163.839996,165.119995,160.869995,164.850006,163.857407,91974200
2022-02-28,163.059998,165.419998,162.429993,165.119995,164.125778,95056600
2022-03-01,164.699997,166.600006,161.970001,163.199997,162.217331,83474400
2022-03-02,164.389999,167.360001,162.949997,166.559998,165.557098,79724800
2022-03-03,168.470001,168.910004,165.550003,166.229996,165.229080,76678400
...,...,...,...,...,...,...
2023-02-17,152.350006,153.000000,150.850006,152.550003,152.550003,59095900
2023-02-21,150.199997,151.300003,148.410004,148.479996,148.479996,58867200
2023-02-22,148.869995,149.949997,147.160004,148.910004,148.910004,51011300


# Finding coefficients for MPT

Here we define the stocks we will optimize. We used two sets of 10 and 20 assets. 10-set was optimized in VQE and quantum-annealing implementation, while 20-set was optimized in quantum-inspired algorithm, where 10-set is too small to catch the difference. This downloaded notebook will show only the results for 20-set.

In [95]:
tickers = ['AAPL', 'MSFT', 'GOOGL', 'CSCO', 'KR', 'LHX', 'IVZ', 'IPGP', 'WMT',  'ABC']
tickers = ['DISH', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE','SEE', 'SRE','AAPL', 'MSFT', 'GOOGL', 'CSCO', 'KR', 'LHX', 'IVZ', 'IPGP', 'WMT',  'ABC']

print(len(tickers))

20


## Data loading
This function downloads the stock data for each of the tickers and changes the format to numpy array. The shape of the full dataset is printed for convinience. 

In [96]:
import numpy as np
def get_stock_data(stock):
    
    """ try to query the iex for a stock, if failed note with print """
    try:
        print(stock)
        stock_df = pdr.get_data_yahoo(stock, start_time, now_time)
        output_name = os.path.join(dirpath, stock) + '.csv'
        stock_df.to_csv(output_name)
        return np.genfromtxt(output_name, delimiter=',')
    except:
        bad_names.append(stock)
        print('bad: %s' % (stock))

data = []
k = 0
for t in tickers:
    data.append(get_stock_data(t))
    k+=1
data = np.array(data)
print(data.shape)

DISH
[*********************100%***********************]  1 of 1 completed
DG
[*********************100%***********************]  1 of 1 completed
DLTR
[*********************100%***********************]  1 of 1 completed
D
[*********************100%***********************]  1 of 1 completed
DPZ
[*********************100%***********************]  1 of 1 completed
DOV
[*********************100%***********************]  1 of 1 completed
DOW
[*********************100%***********************]  1 of 1 completed
DTE
[*********************100%***********************]  1 of 1 completed
SEE
[*********************100%***********************]  1 of 1 completed
SRE
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
MSFT
[*********************100%***********************]  1 of 1 completed
GOOGL
[*********************100%***********************]  1 of 1 completed
CSCO
[*********************100%*****************

Here we remove text data, such as dates and coulumn labels.

In [97]:
data = data[:, 1:data.shape[1], 1:data.shape[2]]
data.shape

(20, 251, 6)

## Daily price

Here we calculate the relative return for each day for each of the assets.

In [98]:
MPT = np.zeros((data.shape[0], data.shape[1]))
for p in range(data.shape[0]):
    for j in range(data.shape[1]):
        MPT[p, j] = (data[p][j][3]-data[p][j][0])/data[p][j][3]
print(MPT.shape)

(20, 251)


Computing expected return as a mean value of returns. Computing risk as a covariance matrix of our assets. The resulting coefficients are valid for classic QUBO formulation of MPT, so that the solution consists of elements that are 0 and 1. 

In [99]:
R = MPT.mean(axis=1)
C = np.cov(MPT)   

This are coefficients for MPT optimization problem, where string solution is s_i = {0,1}. We need to move to new variables as x_i = {-1,1} either -1 or +1. For this we do replacement of x_i = 2s_i - 1, and rewrite resulting coefficients.

## As a result, we obtain J and h
$\sum_{i,j}J_{i,j}x_ix_j - \sum_i h_ix_i $

In [101]:
h_s = C.sum(axis = 1)
h = (h_s - R)/2
J = np.array(C)/4
print('h =', list(h))
print('J = [')
for i in range(20):
    print(list(J[i]),',')
print(']')

h = [0.0036346112659595906, 0.0006316909776104469, 0.0008699437510284579, 0.0013392671856833458, 0.0013879278037356867, 0.0012141871119801435, 0.0008889385674761964, 0.0011553376000349692, 0.0017615481545932732, 0.0008911406360305534, 0.0011716979538296292, 0.0015405733396617767, 0.001543641915993737, 0.001570573032459637, 0.0013369236039704004, 0.0011422850749643653, 0.0013750420141347955, 0.001131124498172374, 0.00047791437377213734, 0.0006013557401137384]
J = [
[0.00032046046738773236, 2.9170726800890566e-05, 4.794455109070797e-05, 1.743194222284266e-05, 4.396621124520363e-05, 5.580537021872531e-05, 4.819681199885957e-05, 2.360712380745998e-05, 4.9583783787242055e-05, 2.4848893910628197e-05, 7.345478931018381e-05, 6.633853914320603e-05, 6.379775776751954e-05, 3.952437545893724e-05, 2.5089458214801202e-05, 1.987548443832619e-05, 9.731850745795969e-05, 8.930827918950908e-05, 2.1024675975173295e-05, 1.9355763101615392e-05] ,
[2.9170726800890566e-05, 5.518250394454241e-05, 3.88062689934